# [미션] 서울 아파트 가격 데이터 분석

이번 미션에서는 2022년 서울 아파트 거래 데이터를 분석해보도록 하겠습니다.

In [1]:
import pandas as pd
import numpy as np

데이터셋을 불러오고 확인합니다.

In [12]:
df=pd.read_csv("./data/seoul_apart_2022.csv")

df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987.0,언주로 3,NaN,중개거래,서울 강남구
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987.0,언주로 3,NaN,중개거래,서울 강남구
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987.0,언주로 3,NaN,중개거래,서울 강남구
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,102.32,202204,1,"369,000",13,2020.0,개포로 264,NaN,중개거래,"서울 강남구, 서울 양천구"
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,136.06,202205,2,"420,000",17,2020.0,개포로 264,NaN,중개거래,서울 강남구


전체 데이터의 갯수와 자료형 등을 확인합니다.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       12684 non-null  object 
 1   번지        12681 non-null  object 
 2   본번        12682 non-null  float64
 3   부번        12682 non-null  float64
 4   단지명       12684 non-null  object 
 5   전용면적(㎡)   12684 non-null  float64
 6   계약년월      12684 non-null  int64  
 7   계약일       12684 non-null  int64  
 8   거래금액(만원)  12651 non-null  object 
 9   층         12684 non-null  int64  
 10  건축년도      12682 non-null  float64
 11  도로명       12684 non-null  object 
 12  해제사유발생일   715 non-null    float64
 13  거래유형      12684 non-null  object 
 14  중개사소재지    12684 non-null  object 
dtypes: float64(5), int64(3), object(7)
memory usage: 1.5+ MB


### [TODO] 데이터프레임 `df`에서 사용하지 않을 특정 컬럼을 제거하세요.

* `drop`을 활용해서 데이터 분석 과정에서 활용하기 어려운 컬럼들을 삭제합니다.
* 삭제할 컬럼은 다음과 같습니다: **"해제사유발생일", "중개사소재지", "번지", "본번", "부번", "도로명", "거래유형"**
* 컬럼 삭제가 데이터프레임 `df`에 적용이 되어야 합니다.

In [13]:
columns = ['해제사유발생일','중개사소재지','번지','본번','부번','도로명','거래유형']

df.drop(columns =  columns , inplace = True)

"전용면적(㎡)" 컬럼의 제곱미터가 특수문자라 사용하기 어려움으로 일단 "전용면적"으로 바꾸도록 하겠습니다.

In [14]:
df=df.rename(columns={"전용면적(㎡)":"전용면적"})

"시군구"컬럼의 주소는 활용하기 쉽게 "구" 컬럼과 "동" 컬럼으로 분리합니다.

In [15]:
# 시군구 분리하기
df["구"]=df["시군구"].apply(lambda e: e.split()[1])
df["동"]=df["시군구"].apply(lambda e: e.split()[2])
df.head()

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,구,동
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987.0,강남구,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987.0,강남구,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987.0,강남구,개포동
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,202204,1,"369,000",13,2020.0,강남구,개포동
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,202205,2,"420,000",17,2020.0,강남구,개포동


전용면적이 60 이하면 **소형**, 60보다 크고 85 이하면 **중형**, 85보다 크고 102 이하면 **중대형**, 102보다 크면 **대형**으로 분류됩니다.

In [16]:
# 면적에 따라 아파트 유형을 분류하는 함수
def category(e):
    if e<=60:
        return "소형"
    elif e<=85:
        return "중형"
    elif e<=102:
        return "중대형"
    else:
        return "대형"

df["유형"]=df["전용면적"].apply(category)
df.head()

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987.0,강남구,개포동,중형
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987.0,강남구,개포동,중형
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987.0,강남구,개포동,중형
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,202204,1,"369,000",13,2020.0,강남구,개포동,대형
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,202205,2,"420,000",17,2020.0,강남구,개포동,대형


"계약년월" 컬럼과 "계약일" 컬럼을 합치고, 날짜타입으로 변경합니다.

In [17]:
df["계약일"]=df["계약년월"].astype('str') + df["계약일"].astype(str)
df["계약일"] = pd.to_datetime(df["계약일"], format='%Y%m%d')

df["계약일"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 12684 entries, 0 to 12683
Series name: 계약일
Non-Null Count  Dtype         
--------------  -----         
12684 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 99.2 KB


"계약일" 컬럼의 데이터가 `datetime`으로 변경되었습니다. 이제 계약일 데이터를 활용해 "계약월"과 "계약요일" 컬럼을 생성해보겠습니다.

In [18]:
df["계약월"]=df["계약일"].dt.month
df["계약요일"]=df["계약일"].dt.dayofweek
df["계약요일"]=df["계약요일"].map({0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'})

df.head()

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형,계약월,계약요일
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,2022-04-12,"220,000",4,1987.0,강남구,개포동,중형,4,화
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,2022-04-21,"220,000",2,1987.0,강남구,개포동,중형,4,목
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202205,2022-05-27,"216,000",2,1987.0,강남구,개포동,중형,5,금
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,202204,2022-04-01,"369,000",13,2020.0,강남구,개포동,대형,4,금
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,202205,2022-05-02,"420,000",17,2020.0,강남구,개포동,대형,5,월


부동산 거래 데이터에서 가장 중요한 데이터는 "거래금액(만원)"입니다. 이 거래금액을 활용하여 정렬을 하거나 다양한 통계값을 계산할 수 있습니다.

"거래금액(만원)" 컬럼을 활용하기 전에 데이터의 결측치를 확인해보도록 하겠습니다.

In [30]:
df.isnull().sum()

시군구          0
단지명          0
전용면적         0
계약년월         0
계약일          0
거래금액(만원)    33
층            0
건축년도         2
구            0
동            0
유형           0
계약월          0
계약요일         0
dtype: int64

"거래금액(만원)" 컬럼에 33개의 결측치가 있습니다. 거래금액은 데이터 분석에서 굉장히 중요한 정보이기 때문에 거래금액 정보가 없는 데이터는 사용할 수 없고, 다른 통계값으로 채워넣기에도 좋지 않습니다.

마침 데이터가 12000개가 넘어서 꽤 많기 때문에, 이번에는 "거래금액(만원)"컬럼에 결측치가 있는 데이터는 아예 삭제해버리도록 하겠습니다.

### [TODO] 데이터프레임 `df`에서 "거래금액(만원)"컬럼에 결측치가 존재하는 데이터를 삭제하세요.

* `dropna`를 활용해서 "거래금액(만원)" 컬럼에 결측치가 존재하는 아파트 거래 데이터를 삭제하세요.
* 데이터 삭제 후 `dropna`의 `ignore_index`를 올바르게 설정하여 인덱스를 초기화해주어야 합니다.

In [24]:
df

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형,계약월,계약요일
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,2022-04-12,"220,000",4,1987.0,강남구,개포동,중형,4,화
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202204,2022-04-21,"220,000",2,1987.0,강남구,개포동,중형,4,목
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,202205,2022-05-27,"216,000",2,1987.0,강남구,개포동,중형,5,금
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,202204,2022-04-01,"369,000",13,2020.0,강남구,개포동,대형,4,금
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,202205,2022-05-02,"420,000",17,2020.0,강남구,개포동,대형,5,월
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,202203,2022-03-27,"73,000",20,1997.0,중랑구,중화동,소형,3,일
12680,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,202207,2022-07-20,"74,000",3,1997.0,중랑구,중화동,소형,7,수
12681,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,202207,2022-07-27,"91,500",12,1997.0,중랑구,중화동,중형,7,수
12682,서울특별시 중랑구 중화동,한영(101),84.69,202204,2022-04-09,"49,900",7,2003.0,중랑구,중화동,중형,4,토


In [34]:
df = df.dropna(subset = '거래금액(만원)', axis = 0, ignore_index=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12651 entries, 0 to 12650
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   시군구       12651 non-null  object        
 1   단지명       12651 non-null  object        
 2   전용면적      12651 non-null  float64       
 3   계약년월      12651 non-null  int64         
 4   계약일       12651 non-null  datetime64[ns]
 5   거래금액(만원)  12651 non-null  object        
 6   층         12651 non-null  int64         
 7   건축년도      12649 non-null  float64       
 8   구         12651 non-null  object        
 9   동         12651 non-null  object        
 10  유형        12651 non-null  object        
 11  계약월       12651 non-null  int32         
 12  계약요일      12651 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(7)
memory usage: 1.2+ MB


데이터가 33개 줄어들기는 했지만, 결측치가 있는 데이터가 삭제되었습니다. 

`info`를 통해 데이터의 정보를 다시 확인해보면 "거래금액(만원)"컬럼의 경우 데이터타입이 `object`인 것을 확인할 수 있습니다. 원활한 데이터 분석을 위해 콤마(,)를 삭제하고 숫자형태 데이터로 바꾸어주도록 하겠습니다.

In [36]:
df["거래금액(만원)"]=df["거래금액(만원)"].str.replace(",","") # str.replace를 활용해 콤마(,)를 아무것도 없는() 것으로 교체
df["거래금액(만원)"] = pd.to_numeric(df["거래금액(만원)"])

df["거래금액(만원)"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 12651 entries, 0 to 12650
Series name: 거래금액(만원)
Non-Null Count  Dtype
--------------  -----
12651 non-null  int64
dtypes: int64(1)
memory usage: 99.0 KB


/var/folders/3_/_16tlvps5z768_sdb6hpw9q80000gn/T/ipykernel_25465/2423137483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["거래금액(만원)"]=df["거래금액(만원)"].str.replace(",","") # str.replace를 활용해 콤마(,)를 아무것도 없는() 것으로 교체
/var/folders/3_/_16tlvps5z768_sdb6hpw9q80000gn/T/ipykernel_25465/2423137483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["거래금액(만원)"] = pd.to_numeric(df["거래금액(만원)"])


"거래금액(만원)" 컬럼의 데이터 타입이 정수형으로 바뀐 것을 확인할 수 있습니다. 이제 거래금액을 기준으로 내림차순으로 정렬해보도록 하겠습니다.

In [37]:
df.sort_values("거래금액(만원)", ascending=False).head(10)

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형,계약월,계약요일
762,서울특별시 강남구 청담동,PH129,273.9600,202204,2022-04-28,1450000,16,2020.0,강남구,청담동,대형,4,목
11098,서울특별시 용산구 한남동,파르크한남,268.6700,202204,2022-04-30,1350000,5,2020.0,용산구,한남동,대형,4,토
8308,서울특별시 성동구 성수동1가,아크로서울포레스트,264.5460,202209,2022-09-30,1300000,47,2020.0,성동구,성수동1가,대형,9,금
11103,서울특별시 용산구 한남동,한남더힐,240.3050,202205,2022-05-30,1100000,3,2011.0,용산구,한남동,대형,5,월
11085,서울특별시 용산구 한남동,나인원한남,206.8953,202211,2022-11-07,945000,8,2019.0,용산구,한남동,대형,11,월
11086,서울특별시 용산구 한남동,르가든더메인한남,225.4100,202205,2022-05-26,900000,6,2019.0,용산구,한남동,대형,5,목
11087,서울특별시 용산구 한남동,르가든더메인한남,269.1200,202206,2022-06-02,900000,2,2019.0,용산구,한남동,대형,6,목
11106,서울특별시 용산구 한남동,한남더힐,235.3120,202207,2022-07-21,890000,3,2011.0,용산구,한남동,대형,7,목
8295,서울특별시 성동구 성수동1가,갤러리아포레,217.8600,202206,2022-06-03,880000,42,2011.0,성동구,성수동1가,대형,6,금
352,서울특별시 강남구 도곡동,타워팰리스1,301.4700,202205,2022-05-12,870000,56,2002.0,강남구,도곡동,대형,5,목


이번엔 "거래금액(만원)" 컬럼의 평균값과 중앙값을 계산해보겠습니다.

In [38]:
print("거래금액 평균값(만원): ", df["거래금액(만원)"].mean())
print("거래금액 중앙값(만원): ", df["거래금액(만원)"].median())

거래금액 평균값(만원):  97562.1373013991
거래금액 중앙값(만원):  75000.0


아파트의 거래금액 자체도 중요하지만 크기에 따른 가격 또한 매우 중요한 지표입니다.

"전용면적"컬럼의 단위는 제곱미터로, 이는 국제 표준 규격이지만 우리에게는 평 단위가 조금 더 익숙합니다. 1평은 대략 3.3㎡ 이므로, "전용면적"컬럼의 값을 3.3으로 나누어 덮어씌우고 컬럼의 이름을 "전용면적(평)"으로 변경합니다.

In [39]:
df["전용면적"] = round(df["전용면적"]/3.3, 2) # round(,2)는 소수점 두번째 자리에서 반올림
df=df.rename(columns={"전용면적":"전용면적(평)"})

df.head()

/var/folders/3_/_16tlvps5z768_sdb6hpw9q80000gn/T/ipykernel_25465/3502211382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["전용면적"] = round(df["전용면적"]/3.3, 2) # round(,2)는 소수점 두번째 자리에서 반올림


,시군구,단지명,전용면적(평),계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형,계약월,계약요일
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,24.23,202204,2022-04-12,220000,4,1987.0,강남구,개포동,중형,4,화
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,24.23,202204,2022-04-21,220000,2,1987.0,강남구,개포동,중형,4,목
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,24.23,202205,2022-05-27,216000,2,1987.0,강남구,개포동,중형,5,금
3,서울특별시 강남구 개포동,개포래미안포레스트,31.01,202204,2022-04-01,369000,13,2020.0,강남구,개포동,대형,4,금
4,서울특별시 강남구 개포동,개포래미안포레스트,41.23,202205,2022-05-02,420000,17,2020.0,강남구,개포동,대형,5,월


이제 "전용면적(평)"과 "거래금액(만원)" 컬럼을 활용해 "평당금액" 컬럼을 생성해보도록 하겠습니다.

### [TODO] 데이터프레임 `df`에 "평당금액" 컬럼을 생성하세요.

* 시리즈 연산을 활용한 데이터 변환을 통해 `df`에 "평당금액" 컬럼을 생성하세요.
* 평당금액은 **"거래금액(만원)"값을 "전용면적(평)"값으로 나눈 값**입니다.

In [40]:
df['평당금액'] = df['거래금액(만원)'] / df['전용면적(평)']

In [41]:
df["평당금액"]=round(df["평당금액"],2) # 소수점 둘째자리 반올림
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12651 entries, 0 to 12650
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   시군구       12651 non-null  object        
 1   단지명       12651 non-null  object        
 2   전용면적(평)   12651 non-null  float64       
 3   계약년월      12651 non-null  int64         
 4   계약일       12651 non-null  datetime64[ns]
 5   거래금액(만원)  12651 non-null  int64         
 6   층         12651 non-null  int64         
 7   건축년도      12649 non-null  float64       
 8   구         12651 non-null  object        
 9   동         12651 non-null  object        
 10  유형        12651 non-null  object        
 11  계약월       12651 non-null  int32         
 12  계약요일      12651 non-null  object        
 13  평당금액      12651 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(3), object(6)
memory usage: 1.3+ MB


이제 평당금액을 기준으로 다시 내림차순 정렬해보도록 하겠습니다.

In [42]:
df.sort_values("평당금액", ascending=False).head(10)

,시군구,단지명,전용면적(평),계약년월,계약일,거래금액(만원),층,건축년도,구,동,유형,계약월,계약요일,평당금액
8144,서울특별시 서초구 잠원동,아크로리버뷰신반포,23.79,202206,2022-06-24,438000,26,2018.0,서초구,잠원동,중형,6,금,18411.10
7584,서울특별시 서초구 반포동,반포주공1단지,32.57,202206,2022-06-29,590000,3,1973.0,서초구,반포동,대형,6,수,18114.83
7602,서울특별시 서초구 반포동,아크로리버파크,25.74,202201,2022-01-21,466000,8,2016.0,서초구,반포동,중형,1,금,18104.12
762,서울특별시 강남구 청담동,PH129,83.02,202204,2022-04-28,1450000,16,2020.0,강남구,청담동,대형,4,목,17465.67
7583,서울특별시 서초구 반포동,반포주공1단지,32.57,202206,2022-06-20,565000,5,1973.0,서초구,반포동,대형,6,월,17347.25
7610,서울특별시 서초구 반포동,아크로리버파크,39.38,202205,2022-05-23,680000,19,2016.0,서초구,반포동,대형,5,월,17267.65
7586,서울특별시 서초구 반포동,반포주공1단지,42.46,202209,2022-09-06,730000,3,1973.0,서초구,반포동,대형,9,화,17192.65
7608,서울특별시 서초구 반포동,아크로리버파크,25.75,202204,2022-04-13,440000,22,2016.0,서초구,반포동,중형,4,수,17087.38
8145,서울특별시 서초구 잠원동,아크로리버뷰신반포,23.79,202207,2022-07-20,405000,11,2018.0,서초구,잠원동,중형,7,수,17023.96
7577,서울특별시 서초구 반포동,반포주공1단지,32.20,202204,2022-04-22,545000,1,1973.0,서초구,반포동,대형,4,금,16925.47


앞서 거래금액을 기준으로 정렬했던결과 많이 다른 결과를 얻을 수 있습니다. 이러한 결과를 부동산 도메인 지식과 함께 활용한다면 유용한 분석이 이루어질 수 있을 것입니다. 이렇게 새로운 지표로 이루어진 컬럼을 생성하는 것은 데이터 분석 과정에서 매우 중요합니다.

이번엔 "구"컬럼을 기준으로 묶어 서울의 각 구별 평당금액의 평균값을 확인해보도록 하겠습니다.

In [43]:
df.groupby("구")[["평당금액"]].mean()

,평당금액
구,
강남구,7575.502330
강동구,4363.266661
강북구,3106.891934
강서구,3417.152295
관악구,3448.876388
광진구,4702.455439
구로구,2904.620233
금천구,3349.048969
노원구,3686.734916


### [TODO] 서초구의 동 중 **평당금액** 평균이 2번째로 높은 동의 이름을 입력하세요.
* `groupby`와 "구", "동" 컬럼을 활용하여 서초구의 각 동별 **평당금액 평균값**을 구할 수 있습니다.
* `groupby`를 통해 출력된 결과를 확인하고 평당금액 평균값이 2번째로 높은 동의 이름을 문자열 형태로 변수 `ans1`에 저장하세요.
* ex) 정답이 망원동일 경우 -> `ans1 = "망원동"`

In [62]:
df.groupby(['구','동'])['평당금액'].mean()['서초구'].sort_values(ascending=False)

동
반포동    12392.411218
잠원동    10398.817979
내곡동     6149.430000
신원동     6121.855000
방배동     5595.841667
서초동     5454.335684
우면동     5203.296296
양재동     4092.363846
Name: 평당금액, dtype: float64

In [63]:
# None을 지우고 알맞은 답을 입력하세요.
ans1='잠원동'